# Example of delayedPLIER repository 

Running the example in the DelayedPLIER repository, I submitted a PR that changes the `setAutoRealizationBackend` function to solve the problem.

# Load libraries/modules

In [4]:
library(TENxPBMCData)
library(here)

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    

# Load data

In [12]:
# define output nb
output_nb_path = here('output/nbs/example_of_delayedPLIER_repository')
dir.create(output_nb_path, showWarnings = FALSE)

sce <- TENxPBMCData("pbmc4k")
mat <- counts(sce)
rownames(mat) <- rowData(sce)$ENSEMBL_ID 
colnames(mat) <- colData(sce)$Sequence

hdf5_count = file.path(output_nb_path, 'counts.hdf5')
hdf5_dimnames = file.path(output_nb_path, 'dimnames.RDS')

writeHDF5Array(mat, filepath = hdf5_count, name = "count")
saveRDS(list(row.names = rowData(sce)$ENSEMBL_ID , col.names = colData(sce)$Sequence), file = hdf5_dimnames)

path_script_funcs = '/home/msubirana/Documents/pivlab/DelayedPLIER/funcs.R'
priorMat <- readRDS('/home/msubirana/Documents/pivlab/DelayedPLIER/canonicalPathways_ENSG.RDS')

see ?TENxPBMCData and browseVignettes('TENxPBMCData') for documentation

loading from cache

Can not create dataset. Object with name 'count' already exists.



ERROR: Error in H5Dcreate(loc$H5Identifier, dataset, tid, sid, dcpl = dcpl): HDF5. Dataset. Unable to initialize object.


In [34]:
head(mat)

<6 x 4340> sparse DelayedMatrix object of type "integer":
                AAACCTGAGAAGGCCT ... TTTGTCATCCCAAGAT
ENSG00000243485                0   .                0
ENSG00000237613                0   .                0
ENSG00000186092                0   .                0
ENSG00000238009                0   .                0
ENSG00000239945                0   .                0
ENSG00000239906                0   .                0

In [7]:
source(path_script_funcs) #glmnet and HDF5Array will be loaded 
setAutoRealizationBackend("HDF5Array") #supportedRealizationBackends(), getRealizationBackend()

Loaded glmnet 4.1-8



In [9]:
sce <- DelayedArray(seed = HDF5ArraySeed(filepath = hdf5_count, name = "count"))

In [10]:
dimnamaes <- readRDS(hdf5_dimnames)
rownames(sce) <- dimnamaes$row.names
colnames(sce) <- dimnamaes$col.names

In [11]:
sce[is.na(sce)] <- 0
data <- sce[which(DelayedMatrixStats::rowSds(sce) >0),]

In [14]:
ptm <- proc.time()
output_dplier = file.path(output_nb_path, 'dplier_result')
dir.create(output_dplier, showWarnings = FALSE)
PLIER.res <- PLIER(data, priorMat, output_path = output_dplier)
print(proc.time()-ptm)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'nrow': object 'priorMat' not found


In [ ]:
head(data.frame(PLIER.res$B))

In [ ]:
head(data.frame(PLIER.res$Z))

In [ ]:
head(data.frame(PLIER.res$U))